### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import os
import pravega.grpc_gateway as pravega
from itertools import islice

In [ ]:
import pravega.video as video
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer, ImageFileSequenceLoader
imp.reload(video);

### Define Pravega stream parameters

In [ ]:
#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
#gateway = '10.243.37.108:80'
scope = 'examples'
input_stream_name = 'multi-video-grid-input'
output_stream_name = 'multi-video-grid-output'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### Create Pravega stream that will be input to Multi Video Grid Job

In [ ]:
input_stream = IndexedStream(pravega_client, scope, input_stream_name)

In [ ]:
# input_stream.delete_stream()
input_stream.create_stream()
input_stream.truncate_stream()

### Create Pravega stream that will be output from Multi Video Grid Job

In [ ]:
output_stream = OutputStream(pravega_client, scope, output_stream_name)

In [ ]:
# output_stream.delete_stream()
output_stream.create_stream()
output_stream.truncate_stream()

### Ingest JPEG images from files

In [ ]:
data_dir = '../../../data/video/'
camera_filespecs = [
    data_dir + 'drone/DJI_0023/*.jpg',
    data_dir + 'virat/aerial/09152008flight2tape1_10/*.jpg',
    data_dir + 'virat/ground/VIRAT_S_000200_01_000226_000268/*.jpg',    
    data_dir + 'virat/ground/VIRAT_S_000201_08_001652_001838/*.jpg',
    
    data_dir + 'drone/DJI_0039/*.jpg',
    data_dir + 'meva/uav-drop-01/2018-03-13/17/2018-03-13.17-30-58.17-40-14.uav1/*.jpg',
    data_dir + 'virat/aerial/09172008flight1tape1_5/*.jpg',
    data_dir + 'virat/aerial/09152008flight2tape3_1/*.jpg',
]
fps = 10  # drone is 24, virat is 30
step = 3

In [ ]:
loader = ImageFileSequenceLoader(scope, input_stream_name, camera_filespecs, fps)

In [ ]:
events_to_write = loader.event_generator()
input_stream.write_events(events_to_write)

### Run Multi Video Grid Flink Job while above is writing events

...